In [141]:
import polars as pl
import os

pl.Config.set_tbl_cols(-1)          # Show all columns
pl.Config.set_tbl_rows(-1)          # Show all rows
pl.Config.set_fmt_str_lengths(100)  # Full string content
pl.Config.set_tbl_width_chars(1000) # Wide display

workdir = os.path.join(os.getcwd(), "source")

source_data_df = pl.read_csv(
    os.path.join(workdir, "apontamentos.csv"),
    encoding='utf-8',
    infer_schema_length=10000
)

source_data_df.head(1)

source_data_df = source_data_df.select(
    pl.col("Maquina\nBWMANR").alias("maquina"),
    pl.col("Numero da Peça\nBWTENR").alias("peca_produzida"),
    pl.col("Turno").alias("turno"),
    pl.col("TM\nAGMAZT").alias("tm"),
    pl.col("Qtde Produzida Boa\nBWRMMG").alias("qtde_produzida"),
    pl.col("Data_ini").alias("data_inicio"),
    pl.col("Hora_ini").alias("hora_inicio"),
    pl.col("Data_fim").alias("data_fim"),
    pl.col("Hora_fim").alias("hora_fim"),
    pl.col("Unterbrechungsgrund\nBWUGRU").alias("code")
)

source_data_df = source_data_df.with_columns([
    pl.col(col).str.strip_chars() for col in ["maquina", "turno", "peca_produzida"]
])

source_data_df = source_data_df.with_columns([
    pl.col("qtde_produzida")
      .str.replace(",", ".")            # Replace comma with dot (decimal separator)
      .str.replace_all(r"\.", "")      # Remove dots (thousand separators)
      .cast(pl.Float64)                 # Convert to float
      .truediv(100)
])

# mid_df = source_data_df.with_columns([
#     pl.col("data_inicio").str.to_date("%d/%m/%Y"),
#     pl.col("data_fim").str.to_date("%d/%m/%Y")
# ])

mid_df = source_data_df.with_columns([
    # Date columns
    pl.col("data_inicio").str.to_date("%d/%m/%Y"),
    pl.col("data_fim").str.to_date("%d/%m/%Y"),
    
    # Datetime columns (combine date + time)
    (pl.col("data_inicio").str.strip_chars('"') + " " + pl.col("hora_inicio").str.strip_chars('"'))
        .str.to_datetime("%d/%m/%Y %H:%M:%S")
        .alias("inicio_dt"),
    (pl.col("data_fim").str.strip_chars('"') + " " + pl.col("hora_fim").str.strip_chars('"'))
        .str.to_datetime("%d/%m/%Y %H:%M:%S")
        .alias("fim_dt")
])

mid_df = mid_df.with_columns([
    pl.when(pl.col("inicio_dt").dt.hour() < 5)
        .then(pl.col("inicio_dt").dt.date() - pl.duration(days=1))
        .otherwise(pl.col("inicio_dt").dt.date())
        .alias("data_trabalho")
])

df = mid_df.filter(pl.col("peca_produzida") == "110900180")

In [146]:
df

maquina,peca_produzida,turno,tm,qtde_produzida,data_inicio,hora_inicio,data_fim,hora_fim,code,inicio_dt,fim_dt,data_trabalho
str,str,str,f64,f64,date,str,date,str,str,datetime[μs],datetime[μs],date
"""00114""","""110900180""","""Segundo Turno""",0.71,0.0,2025-12-10,"""14:00:00""",2025-12-10,"""17:50:34""","""25""",2025-12-10 14:00:00,2025-12-10 17:50:34,2025-12-10
"""00114""","""110900180""","""Segundo Turno""",0.71,18000.0,2025-12-10,"""19:55:16""",2025-12-10,"""22:05:58""","""03""",2025-12-10 19:55:16,2025-12-10 22:05:58,2025-12-10
"""00114""","""110900180""","""Terceiro Turno""",0.71,11000.0,2025-12-11,"""03:35:50""",2025-12-11,"""04:56:07""","""03""",2025-12-11 03:35:50,2025-12-11 04:56:07,2025-12-10
"""00114""","""110900180""","""Primeiro Turno""",0.71,70000.0,2025-12-11,"""05:16:41""",2025-12-11,"""13:47:59""","""03""",2025-12-11 05:16:41,2025-12-11 13:47:59,2025-12-11
"""00114""","""110900180""","""Segundo Turno""",0.71,36031.0,2025-12-11,"""14:00:00""",2025-12-11,"""21:27:25""","""03""",2025-12-11 14:00:00,2025-12-11 21:27:25,2025-12-11


In [155]:
df_grouped_v1 = df.group_by([
    "turno",
    "peca_produzida", 
    "maquina",
    "data_trabalho",
    "tm"
]).agg([
    pl.col("qtde_produzida").sum().alias("qtde_produzida_total")
]).sort(["data_trabalho", "turno"])

3600 / ((0.71 / 100) * 60)

df_grouped_v1 = df_grouped_v1.with_columns(
    (3600 / ((pl.col("tm") / 100) * 60)).alias("pecas_hora")
)

df_grouped_v1 = df_grouped_v1.with_columns(
    pl.when(pl.col("turno") == "Primeiro Turno").then(7.42)
      .when(pl.col("turno") == "Segundo Turno").then(7.25)
      .when(pl.col("turno") == "Terceiro Turno").then(5.97)
      .otherwise(0.0)
      .alias("horas_turno")
)

df_grouped_v1 = df_grouped_v1.with_columns(
    (pl.col("pecas_hora") * pl.col("horas_turno")).alias("pecas_planejadas")
)


df_grouped_v1

turno,peca_produzida,maquina,data_trabalho,tm,qtde_produzida_total,pecas_hora,horas_turno,pecas_planejadas
str,str,str,date,f64,f64,f64,f64,f64
"""Segundo Turno""","""110900180""","""00114""",2025-12-10,0.71,18000.0,8450.704225,7.25,61267.605634
"""Terceiro Turno""","""110900180""","""00114""",2025-12-10,0.71,11000.0,8450.704225,5.97,50450.704225
"""Primeiro Turno""","""110900180""","""00114""",2025-12-11,0.71,70000.0,8450.704225,7.42,62704.225352
"""Segundo Turno""","""110900180""","""00114""",2025-12-11,0.71,36031.0,8450.704225,7.25,61267.605634


In [150]:
3600 / ((0.71 / 100) * 60)

8450.704225352112

In [111]:
mid_df

maquina,peca_produzida,turno,tm,qtde_produzida,data_inicio,hora_inicio,data_fim,hora_fim,code
str,str,str,f64,f64,str,str,str,str,str
"""00100 ""","""114200005 ""","""Primeiro Turno """,1.33,870000.0,"""10/12/2025""","""07:28:00""","""10/12/2025""","""12:00:00""","""03"""
"""00100 ""","""114200005 ""","""Primeiro Turno """,1.33,150000.0,"""10/12/2025""","""13:10:00""","""10/12/2025""","""13:59:00""","""03"""
"""00100 ""","""114200005 ""","""Segundo Turno """,1.33,1.95e6,"""10/12/2025""","""14:00:00""","""10/12/2025""","""22:04:48""","""03"""
"""00100 ""","""114200005 ""","""Primeiro Turno """,1.33,213800.0,"""11/12/2025""","""07:45:49""","""11/12/2025""","""09:17:22""","""03"""
"""00100 ""","""114200005 ""","""Primeiro Turno """,1.33,0.0,"""11/12/2025""","""09:17:35""","""11/12/2025""","""09:20:46""","""03"""
"""00100 ""","""111900583 ""","""Segundo Turno """,1.33,0.0,"""11/12/2025""","""13:46:00""","""11/12/2025""","""21:25:59""","""RE"""
"""00107 ""","""119900816 ""","""Primeiro Turno """,0.54,0.0,"""11/12/2025""","""05:54:43""","""11/12/2025""","""05:55:26""","""03"""
"""00107 ""","""119900816 ""","""Primeiro Turno """,0.54,0.0,"""10/12/2025""","""05:37:32""","""10/12/2025""","""06:40:34""","""RE"""
"""00107 ""","""119900816 ""","""Segundo Turno """,0.54,0.0,"""11/12/2025""","""19:18:38""","""11/12/2025""","""21:02:18""","""RE"""


In [79]:
mid_df.head(50)

maquina,peca_produzida,turno,tm,qtde_produzida,data_inicio,hora_inicio,data_fim,hora_fim,code,inicio_dt,fim_dt,data_trabalho
i64,str,str,f64,f64,date,str,date,str,str,datetime[μs],datetime[μs],date
170,"""911200010""","""PrimeiroTurno""",6.67,6500.0,2025-11-14,"""05:15:55""",2025-11-14,"""13:28:30""","""03""",2025-11-14 05:15:55,2025-11-14 13:28:30,2025-11-14
170,"""911200010""","""SegundoTurno""",6.67,6000.0,2025-11-14,"""13:53:28""",2025-11-14,"""21:48:00""","""03""",2025-11-14 13:53:28,2025-11-14 21:48:00,2025-11-14
171,"""111900583""","""PrimeiroTurno""",12.0,4379.0,2025-11-14,"""05:14:57""",2025-11-14,"""11:31:55""","""40""",2025-11-14 05:14:57,2025-11-14 11:31:55,2025-11-14
173,"""112001410A""","""PrimeiroTurno""",11.76,3100.0,2025-11-14,"""05:16:27""",2025-11-14,"""13:29:56""","""03""",2025-11-14 05:16:27,2025-11-14 13:29:56,2025-11-14
173,"""112001410A""","""SegundoTurno""",11.76,3000.0,2025-11-14,"""13:53:06""",2025-11-14,"""21:48:35""","""03""",2025-11-14 13:53:06,2025-11-14 21:48:35,2025-11-14
175,"""111600813""","""PrimeiroTurno""",5.56,6400.0,2025-11-14,"""05:16:50""",2025-11-14,"""13:32:09""","""03""",2025-11-14 05:16:50,2025-11-14 13:32:09,2025-11-14
175,"""111600813""","""SegundoTurno""",5.56,5500.0,2025-11-14,"""13:52:15""",2025-11-14,"""21:49:12""","""03""",2025-11-14 13:52:15,2025-11-14 21:49:12,2025-11-14
176,"""112001410A""","""PrimeiroTurno""",11.76,3400.0,2025-11-14,"""05:17:13""",2025-11-14,"""13:31:30""","""03""",2025-11-14 05:17:13,2025-11-14 13:31:30,2025-11-14
176,"""112001410A""","""SegundoTurno""",11.76,3200.0,2025-11-14,"""13:52:39""",2025-11-14,"""21:48:56""","""03""",2025-11-14 13:52:39,2025-11-14 21:48:56,2025-11-14
